<a href="https://colab.research.google.com/github/pascal-bergeron/ai4forgood2016/blob/main/FeatureBuilding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
cp 'drive/My Drive/Projects/AIForGood2020/' . -r

In [15]:
cp '../drive/My Drive/Projects/AIForGood2020/organisation.xlsx' .

In [5]:
cd AIForGood2020/

/content/AIForGood2020


In [6]:
# !unrar e -p*hack20 211GrandMontreal_DataPack.rar

# Agencies Taxonomomies

In [16]:
org = pd.read_excel('organisation.xlsx')

In [17]:
org.set_index('ResourceAgencyNum', inplace=True)

In [18]:
org.tail()

,PublicName,LegalStatus,AlternateName,TaxonomyLevelName,ParentAgency,ParentAgencyNum,LastVerifiedOn,LanguageOfRecord,Custom_Statut temporaire Covid-19,Custom_Acronym,Municipalité_Code,PhysicalAddress1,PhysicalAddressIsPrivate,PhysicalStateProvince,PhysicalPostalCode,Latitude,Longitude,Coverage,TaxonomyCodes
ResourceAgencyNum,,,,,,,,,,,,,,,,,,,
30790854,BIBLIOTHÈQUE PUBLIQUE DE DOLLARD-DES-ORMEAUX,organisme municipal,NaN,Program,DOLLARD-DES-ORMEAUX - VILLE,18760983,2020-06-04,fr,Ouvert,NaN,66142.0,"12001, boulevard De Salaberry",No,QC,H9B 2A7,45.48414,-73.80695,QC - Montréal County - Dollard-Des Ormeaux;,TJ-4400; TJ-1800.6650; TJ-4500.2900; TJ-4500.1...
51229060,BIBLIOTHÈQUE MUNICIPALE DE NAPIERVILLE,organisme municipal,NaN,Program,NAPIERVILLE - MUNICIPALITÉ,19499426,2020-06-05,fr,Ouvert,NaN,68030.0,"290, rue Saint-Alexandre",No,QC,J0J 1L0,45.18862,-73.40354,QC - Montérégie County - Saint-Cyprien-de-Napi...,TJ-4400; TJ-1800.6650; TJ-4500.1500-800; TJ-45...
49492168,BIBLIOTHÈQUE MUNICIPALE DE SAINT-JACQUES-LE-MI...,organisme municipal,NaN,Program,SAINT-JACQUES-LE-MINEUR - MUNICIPALITÉ,19533005,2019-08-21,fr,Ouvert,NaN,68040.0,"89, rue Principale",No,QC,J0J 1Z0,45.27682,-73.41787,QC - Montérégie County - Saint-Jacques-le-Mineur;,TJ-4400; TJ-1800.6650
18766848,"BOISBRIAND - LOISIRS, CULTURE ET BIBLIOTHÈQUE",organisme municipal,NaN,Program,BOISBRIAND - VILLE,18762253,2020-06-08,fr,Ouvert,NaN,73005.0,"940, boulevard de la Grande Allée, rez-de-chau...",No,QC,J7G 2J7,45.61253,-73.83842,QC - Laurentides County - Boisbriand;,TD-0300.1200 * PL-7000.4360
42388216,REBOND (LE),organisme parapublic,NaN,Program,OFFICE MUNICIPAL D'HABITATION DE MONTRÉAL - SI...,18760248,2019-08-14,fr,Ouvert,NaN,NaN,NaN,Yes,QC,NaN,NaN,NaN,QC - Montréal County - Ahuntsic-Cartierville; ...,PS-9800.9800; TJ-1800.3300-600.10 * YB-9500.0500


In [34]:
def get_level_2_taxonomy_code_list(ser: pd.Series) -> pd.Series:
  first_split = ser.str.split("; ")
  return first_split.apply(lambda x : np.nan if isinstance(x, float) and np.isnan(x) else [y[:7] for y in x] )

In [37]:
org['L2_tax_codes'] = get_level_2_taxonomy_code_list(org['TaxonomyCodes'])

In [39]:
org['L2_tax_codes'].head()

ResourceAgencyNum
34515394                                   [TD-6500, TD-1600]
18764735    [TC-5500, YH-6000, ND-3500, HH-0500, HH-0500, ...
18762709                                   [TN-4600, FT-4800]
18765353    [BM-8500, BD-1800, TI-1800, TI-1800, TI-1800, ...
54247598    [TD-1600, PS-8000, RP-1400, PH-0320, RP-1400, ...
Name: L2_tax_codes, dtype: object

# Needs 

In [40]:
besoins = pd.read_excel('Besoins.xlsx')

In [41]:
besoins.set_index('ReportNeedNum', inplace=True)

In [42]:
besoins.tail()

,CallReportNum,TaxonomyCode_full,TaxonomyName_full,NeedWasUnmet,ReasonIfUnmetOrPartial,Level1Code,Level1Name,Level2Code,Level2Name,Level3Code,Level3Name,Level4Code,Level4Name,Level5Code,Level5Name,AIRSNeedCategory,TaxonomyCode_main,TaxonomyCode_link,TaxonomyName_main,TaxonomyName_link
ReportNeedNum,,,,,,,,,,,,,,,,,,,,
21475026,78145762,TI-1800.4500-100,Dons de livres et de magazines,False,NaN,T,"Services organisationnels, communautaires et i...",TI,Services de dons,TI-1800,Dons d'articles spécialisés,TI-1800.4500,Dons d'équipement et fournitures de loisir,TI-1800.4500-100,Dons de livres et de magazines,Bénévolat et dons,TI-1800.4500-100,NaN,Dons de livres et de magazines,NaN
21475411,78146045,NL-3000.1500,Aide pour les frais de garde d'enfants,False,NaN,N,Soutien du revenu et emploi,NL,Aide sociale,NL-3000,Aide sociale relative au maintien à domicile,NL-3000.1500,Aide pour les frais de garde d'enfants,NL-3000.1500,Aide pour les frais de garde d'enfants,Emploi et revenu,NL-3000.1500,NaN,Aide pour les frais de garde d'enfants,NaN
21475365,78146045,PH-6100.1500,Cours de développement de l'enfant,False,NaN,P,Vie personnelle et familiale,PH,Soutien personnel et familial,PH-6100,Éducation parentale,PH-6100.1500,Cours de développement de l'enfant,PH-6100.1500,Cours de développement de l'enfant,"Soutien personnel, familial et communautaire",PH-6100.1500,NaN,Cours de développement de l'enfant,NaN
21475383,78146045,PH-6100.3300,Éducation parentale à domicile,False,NaN,P,Vie personnelle et familiale,PH,Soutien personnel et familial,PH-6100,Éducation parentale,PH-6100.3300,Éducation parentale à domicile,PH-6100.3300,Éducation parentale à domicile,"Soutien personnel, familial et communautaire",PH-6100.3300,NaN,Éducation parentale à domicile,NaN
21475338,78146045,PH-2360.1800,Centres de services à l'enfance et aux familles,False,NaN,P,Vie personnelle et familiale,PH,Soutien personnel et familial,PH-2360,Services familiaux,PH-2360.1800,Centres de services à l'enfance et aux familles,PH-2360.1800,Centres de services à l'enfance et aux familles,"Soutien personnel, familial et communautaire",PH-2360.1800,NaN,Centres de services à l'enfance et aux familles,NaN


In [43]:
len(besoins.TaxonomyCode_full.unique())

1800

In [44]:
len(besoins.Level1Code.unique())

11

In [45]:
len(besoins.Level2Code.unique())

73

In [46]:
len(besoins.Level3Code.unique())

375

# References

In [48]:
references = pd.read_excel('Références.xlsx')

In [49]:
references.tail()

,CallReportNum,ResourceAgencyNum,AgencyNamePublic,ReportNeedNum
264317,78146045,18762631.0,ENTRE MAMANS ET PAPAS,21475383
264318,78146045,18760262.0,UNIS POUR LES PETITS,21475383
264319,78146045,18762552.0,PREMIERS PAS CHAMPLAIN,21475338
264320,78146045,18762552.0,PREMIERS PAS CHAMPLAIN,21475383
264321,78146045,41529174.0,SERVICES QUÉBEC - DEVENIR PARENT,21475411


In [50]:
resource_ref_count= references.groupby('ResourceAgencyNum')['CallReportNum'].count().sort_values(ascending=False)

In [58]:
resource_agency_nums = resource_ref_count[resource_ref_count > 10].index.astype(int).dropna().values
np.sort(resource_agency_nums)

array([18759962, 18759964, 18759965, ..., 68848329, 68933670, 69147049])

In [60]:
resource_agency_nums

array([18760559, 47945224, 18761956, ..., 18762324, 34051134, 18762122])

In [61]:
len(references)

264322

In [62]:
ref = references.dropna()

In [63]:
ref['ResourceAgencyNum_int'] = ref['ResourceAgencyNum'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
ref

,CallReportNum,ResourceAgencyNum,AgencyNamePublic,ReportNeedNum,ResourceAgencyNum_int
0,51805319,18762230.0,SOS VIOLENCE CONJUGALE,12443066,18762230
1,51805954,18760351.0,MISSIONNAIRES DE LA CHARITÉ,12443170,18760351
2,51805954,18762550.0,DÎNERS SAINT-LOUIS,12443170,18762550
3,51806285,18762497.0,CENTRE DE CRISE TRACOM,12443222,18762497
4,51806377,18763063.0,"CENTRE HUMANITAIRE D'ORGANISATION, DE RESSOURC...",12443241,18763063
...,...,...,...,...,...
264317,78146045,18762631.0,ENTRE MAMANS ET PAPAS,21475383,18762631
264318,78146045,18760262.0,UNIS POUR LES PETITS,21475383,18760262
264319,78146045,18762552.0,PREMIERS PAS CHAMPLAIN,21475338,18762552
264320,78146045,18762552.0,PREMIERS PAS CHAMPLAIN,21475383,18762552


In [70]:
tmp = ref[['CallReportNum','ResourceAgencyNum_int']].groupby('CallReportNum')['ResourceAgencyNum_int'].apply(list).values

In [53]:
from sklearn.preprocessing import MultiLabelBinarizer

In [72]:
Y = MultiLabelBinarizer().fit_transform(tmp)

In [73]:
Y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])